In [1]:
import os
import sys
import shutil 
from zipfile import ZipFile
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import skimage
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
import config 
from config import Config
import utils
import model as modellib
#import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "checkpoints.h5")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
    #utils.download_trained_weights(COCO_MODEL_PATH)

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
class Config2(config.Config):

    # Give the configuration a recognizable name
    NAME = "nuclei"
    train_data_root = 'train/' 
    val_data_root = 'train/'
    test_data_root = 'stage1_test/'
    
    MODEL_DIR = 'checkpoints'
    COCO_MODEL_PATH = 'mask_rcnn_coco.h5'
    # imagenet, coco, or last
    init_with = "last"  
    
    LEARNING_RATE = 0.0001
    
    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    
    # Train on 1 GPU and 8 images per GPU. Batch size is GPUs * images/GPU.
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    bs = GPU_COUNT * IMAGES_PER_GPU
    # Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch.
    # typically be equal to the number of samples of your dataset divided by the batch size
    STEPS_PER_EPOCH = 300
    VALIDATION_STEPS = 70//bs

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + nucleis

    # Input image resing
    # Images are resized such that the smallest side is >= IMAGE_MIN_DIM and
    # the longest side is <= IMAGE_MAX_DIM. In case both conditions can't
    # be satisfied together the IMAGE_MAX_DIM is enforced.
    IMAGE_MIN_DIM = 576
    IMAGE_MAX_DIM = 576
   # If True, pad images with zeros such that they're (max_dim by max_dim)
    IMAGE_PADDING = True  # currently, the False option is not supported



    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels, maybe add a 256?
    # The strides of each layer of the FPN Pyramid. These values
    # are based on a Resnet101 backbone.
    BACKBONE = 'resnet101'
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 320 #300
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    # Pooled ROIs
    POOL_SIZE = 7
    MASK_POOL_SIZE = 14
    MASK_SHAPE = [28, 28]
    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # (increasing) the RPN NMS threshold.
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 512
    # Non-max suppression threshold to filter RPN proposals.
    # You can reduce(increase?) this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.6
    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 256
    
    
    # Max number of final detections
    DETECTION_MAX_INSTANCES = 400 
    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.7 # may be smaller?
    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3 # 0.3
    
    
    MEAN_PIXEL = np.array([42.17746161,38.21568456,46.82167803])
    
    # Weight decay regularization
    WEIGHT_DECAY = 0.0001

    
    
opt = Config2()



if __name__ == '__main__':
    opt.display()
    STEPS_PER_EPOCH = 300

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 40
    
    LEARNING_RATE = 0.0001
    
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)
    
    # Max number of final detections
    DETECTION_MAX_INSTANCES = 400 
    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.7 # may be smaller?
    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3 # 0.3
    
    MAX_GT_INSTANCES = 550
    
    
config = opt
config.display()


Configurations:
AUGMENTATION_FLIP_LR           0.5
AUGMENTATION_FLIP_UD           0.5
BACKBONE                       resnet101
BACKBONE_SHAPES                [[144 144]
 [ 72  72]
 [ 36  36]
 [ 18  18]
 [  9   9]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_MODEL_PATH                mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  576
IMAGE_MIN_DIM                  576
IMAGE_PADDING                  True
IMAGE_SHAPE                    [576 576   3]
LEARNING_RATE                  0.0001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               256
MEAN_PIXEL                     [42.17746161 38.21568456 46.82167803]
MINI_MASK_SHAPE                (56, 56)
MODEL_DIR                 

In [3]:
  # Add classes
class NucleiDataset(utils.Dataset):
    
    def load_nuclei(self, root_path, mode='train', filter_ids=None):
        
        self.add_class("nuclei", 1, "nucleus")
        
        if mode == 'inference':
            files = os.listdir(os.path.join(root_path, 'test_imgs'))
        else:
            files = os.listdir(os.path.join(root_path, 'train_imgs'))

        if filter_ids is not None:
            files = [item for item in files if item.split('.')[0] in filter_ids]
            
        for i, ffile in enumerate(tqdm(files)):
            if mode == 'inference':
                data_path = os.path.join(root_path, 'test_imgs', ffile)
                mask_path = 'No masks'
            else:
                data_path = os.path.join(root_path, 'train_imgs', ffile)
                
            bg_color = random.randint(0, 255)
            original_id = ffile.split('.')[0]
            # TODO change later maybe

            data = cv2.imread(data_path)[:, :, 0]
            height, width = data.shape

            self.add_image("nuclei", image_id=i, path=data_path,
                           width=width, height=height,
                           bg_color=bg_color, original_id=original_id,
                           data_path=data_path,
                           root_path=root_path)

            image = cv2.imread(data_path)
            # If grayscale. Convert to RGB for consistency.
            if image.ndim != 3:
                image = skimage.color.gray2rgb(image)
        
    
    def load_image(self, image_id):
        info = self.get_info(image_id)
        path = info['data_path']
        image = cv2.imread(path)
        # If grayscale. Convert to RGB for consistency.
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        return image

    def get_info(self, image_id):
        return self.image_info[image_id]
        
    def load_mask(self, image_id):
        info = self.get_info(image_id)
        original_id = info['original_id']
        root_path = info['root_path']
        width = info['width']
        height = info['height']
        return self.add_mask_data(original_id, root_path, width=width, height=height)[:]

    def add_mask_data(self, original_id, root_path, width=256, height=256):
        # info = self.image_info[image_id]
        orifinal_id = original_id
        root_path = root_path
        
        all_masks = os.listdir(os.path.join(root_path, 'masks'))
        all_masks = [element for element in all_masks if element.split('_')[0] == orifinal_id]

        num_labels = 0
        #mask_ids = next(os.walk(all_masks))[1]
        for i, ffile in enumerate(all_masks): 
            subpath = os.path.join(root_path, 'masks', ffile)
            data = imread(subpath)[:, :]
            if np.sum(data) != 0:
                num_labels += 1

        mask = np.zeros([height,width,num_labels], dtype=np.bool)
        num_labels = 0
        for i, ffile in enumerate(all_masks): 
            subpath = os.path.join(root_path, 'masks', ffile)
            data = imread(subpath)[:, :]

            if np.sum(data) != 0:
                data = (data != 0)
                mask[:, :, num_labels] = data
                num_labels += 1
        
        class_ids = np.array([1] * num_labels)
        class_ids = class_ids.astype(np.int32)
        return mask, class_ids

In [4]:
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
model.load_weights(COCO_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                            "mrcnn_bbox", "mrcnn_mask"])

filter_ids = os.listdir(os.path.join(ROOT_DIR, 'train_imgs'))
files = [item.split('.')[0] for item in filter_ids]
original_id = os.listdir(os.path.join(ROOT_DIR, 'masks'))
all_masks = [element.split('_')[0] for element in original_id]
        


# Training dataset
dataset_train = NucleiDataset()
dataset_train.load_nuclei(ROOT_DIR, mode='training',filter_ids = files[:598] )
dataset_train.prepare()

# Validation dataset
dataset_val = NucleiDataset()
dataset_val.load_nuclei(ROOT_DIR, mode = 'training', filter_ids = files[66:] )
dataset_val.prepare()

100%|██████████| 598/598 [00:04<00:00, 134.01it/s]


In [5]:
print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Image Count: 598
Class Count: 2
  0. BG                                                
  1. nucleus                                           


In [6]:
if opt.init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights('checkpoints.h5/nuclei20180327T2044/mask_rcnn_nuclei_0057.h5', by_name=True)
    

In [8]:
model.train(dataset_train, dataset_val, 
            learning_rate=opt.LEARNING_RATE, 
            epochs=69, 
            layers='all')

"""
#with img_size = 448X448
model.train(dataset_train, dataset_val, 
            learning_rate=opt.LEARNING_RATE, 
            epochs=60, 
            layers='all')
        

model.train(dataset_train, dataset_val, 
            learning_rate=opt.LEARNING_RATE/10, 
            epochs=75, 
            layers='all')
"""



Starting at epoch 58. LR=0.0001

Checkpoint Path: /home/ec2-user/checkpoints.h5/nuclei20180327T2044/mask_rcnn_nuclei_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_

/usr/local/lib/python3.4/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 59/69


/usr/lib64/python3.4/dist-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


300/300 [==============================] - 794s 3s/step - loss: 0.2689 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0452 - mrcnn_class_loss: 0.0212 - mrcnn_bbox_loss: 0.0402 - mrcnn_mask_loss: 0.1601 - val_loss: 0.2964 - val_rpn_class_loss: 9.3338e-05 - val_rpn_bbox_loss: 0.0493 - val_mrcnn_class_loss: 0.0021 - val_mrcnn_bbox_loss: 0.0506 - val_mrcnn_mask_loss: 0.1942
Epoch 60/69
300/300 [==============================] - 706s 2s/step - loss: 0.2769 - rpn_class_loss: 0.0022 - rpn_bbox_loss: 0.0467 - mrcnn_class_loss: 0.0220 - mrcnn_bbox_loss: 0.0429 - mrcnn_mask_loss: 0.1631 - val_loss: 0.3807 - val_rpn_class_loss: 8.1784e-04 - val_rpn_bbox_loss: 0.0996 - val_mrcnn_class_loss: 0.0011 - val_mrcnn_bbox_loss: 0.0658 - val_mrcnn_mask_loss: 0.2135
Epoch 61/69
300/300 [==============================] - 709s 2s/step - loss: 0.2677 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0447 - mrcnn_class_loss: 0.0211 - mrcnn_bbox_loss: 0.0397 - mrcnn_mask_loss: 0.1599 - val_loss: 0.3079 - val_rpn_class_lo

"\n#with img_size = 448X448\nmodel.train(dataset_train, dataset_val, \n            learning_rate=opt.LEARNING_RATE, \n            epochs=60, \n            layers='all')\n        \n\nmodel.train(dataset_train, dataset_val, \n            learning_rate=opt.LEARNING_RATE/10, \n            epochs=75, \n            layers='all')\n"

In [9]:
from skimage import morphology
from skimage.morphology import binary_closing, binary_opening, disk, binary_dilation
from tqdm import tqdm
import pandas as pd
import cv2

def rle_encode(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[1]*shape[0], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape((shape[1], shape[0])).T


dataset = NucleiDataset()
dataset.load_nuclei(ROOT_DIR,'inference')
dataset.prepare()



class InferenceConfig(Config2):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    MEAN_PIXEL = np.array([56.02288505, 54.02376286, 54.26675248])

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=opt.MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = 'checkpoints.h5/nuclei20180327T2044/mask_rcnn_nuclei_0069.h5' #model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "checkpoints.h5/nuclei20180327T2044/mask_rcnn_nuclei_{epoch:04d}.h5"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)



'''
load test dataset one by one. Note that masks are resized (back) in model.detect
rle2csv
'''        
#ImageId = []
#EncodedPixels = []

output = []
sample_submission = pd.read_csv('stage1_sample_submission.csv')

for image_id in tqdm(sample_submission.ImageId):
    image_path = os.path.join('test_imgs', image_id + '.png')
    
    original_image = cv2.imread(image_path)
    results = model.detect([original_image], verbose=0)
    r = results[0]
    
    masks = r['masks']
    
    count = masks.shape[-1]
    occlusion = np.logical_not(masks[:, :, -1]).astype(np.uint8)
    
    for i in range(count - 2, -1, -1):
        mask = masks[:, :, i] * occlusion
        mask_rle = rle_to_string(rle_encode(mask))
        
        # Sanity check
        try:
            rle_decode(mask_rle, original_image.shape[:-1])
            output.append([image_id, mask_rle])
            occlusion = np.logical_and(occlusion, np.logical_not(masks[:, :, i]))
        
        except Exception as e:
            print(e)
            print(image_id)
            print('---')
        
output_df = pd.DataFrame(output, columns=['ImageId', 'EncodedPixels'])
output_df.to_csv('42nd_sub.csv', index=False, encoding='utf-8')

100%|██████████| 65/65 [00:00<00:00, 100.59it/s]


Loading weights from  checkpoints.h5/nuclei20180327T2044/mask_rcnn_nuclei_0069.h5


100%|██████████| 65/65 [01:40<00:00,  1.55s/it] 
